# CrewAI with Ollama

This Jupyter notebook demonstrates the use of CrewAI in conjunction with the Ollama language model. 

The notebook outlines the creation of various agents and tasks for content preparation, specifically for a YouTube channel focused on topics such as AI, Machine Learning, and Neural Networks.

[Video of the Live Session](https://youtu.be/5hzFOYjH6XI)

[CrewAI](https://crewai.com/) | [Ollama](https://ollama.com/) | [LangChain](https://langchain.com/) | [Agentops](https://agentops.ai/)


#### Install the required packages

In [1]:
%%capture
%pip install -U crewai --force-reinstall
%pip langchain -U langsmith langchain langchain_community arxiv wikipedia 
%pip install -U agentops

In [11]:
import logging
logging.basicConfig(filename='interaction_logs.log', level=logging.DEBUG, format='%(asctime)s - %(levelname)s - %(message)s\n')

In [13]:
import os
os.environ['LANGCHAIN_TRACING_V2'] = 'True'
os.environ['LANGSMITH_API_KEY'] = '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'

This cell installs the necessary packages for the notebook, including crewai, langchain, langchain_community, arxiv, wikipedia.

The `%%capture` command suppresses the output of the installation process.

## Tasks

In [21]:
from crewai import Agent, Task
from textwrap import dedent
import os
from langchain_community.chat_models.ollama import ChatOllama
from langchain.prompts import ChatPromptTemplate, PromptTemplate
from langchain_core.output_parsers.string import StrOutputParser
from langchain.callbacks import LangChainTracer
from langsmith import Client


# optional
from agentops.langchain_callback_handler import LangchainCallbackHandler as AgentOpsLangchainCallbackHandler
agentops_handler = AgentOpsLangchainCallbackHandler(api_key=os.environ.get('AGENTOPS_API_KEY'), tags=['CrewAI Example'])

langchain_tracer = LangChainTracer(project_name="CrewAI Example")
from IPython.display import display, Markdown

llm = ChatOllama(model="phi3",
    n_ctx=4096,
    temperature=0.9,
)


In [15]:
# Example chain 
#prompt_contribute = PromptTemplate.from_template("""\
#You are a {Role}; 
#{BackStory}
#Your goal is :{Goal}
#Using {Tools}
#""")
#chain  = prompt_contribute | llm | StrOutputParser()
#chain.invoke(""")


This cell imports the necessary libraries and modules. 

These include CrewAI's Agent and Task, dedent from textwrap, various components from langchain, and display utilities from IPython.

In [22]:

class ContentPreparationTasks():
    def research_task(self, agent, topics, subject):
        return Task(
            description=dedent(f"""\
Conduct comprehensive research on each topic already covered by my channel. Gather information on recent
news, trends and research papers about the Channel Topic.
Identify potential gaps and new directions for future content.
                        
                Topics: {topics}
                Channel Subject: {subject}"""),
            expected_output=dedent("""\
A detailed report summarizing new topics and directions to consider
for new content relevant to the channel subject."""),
            async_execution=False,
            agent=agent, 
        )

    def outline_creation_task(self, agent, new_topic, subject):
        return Task(
            description=dedent(f"""\
Analyze the current industry trends, challenges, and opportunities
relevant to the channel subject. Consider research papers, reports, recent
developments, and expert opinions to provide a comprehensive
overview of the {new_topic} for my channel about: {subject}.
Identify major trends, potential challenges, and future advancements."""),
            expected_output=dedent(f"""\
An insightful and engaging outline for content on my channel about '{subject}' that identifies major trends, potential
challenges, and future advancements about {new_topic}."""),
            async_execution=False,
            agent=agent, 
        )
	
    def cross_reference_content_task(self, agent):
        return Task(
            description=dedent("""\
Analyze the current topics and the outline of the topic to create a comprehensive
list of cross-references, and connections between the new topics and the existing ones.
This will help in creating a cohesive and well-structured narrative.

## Topics
{topics}

## New Topic
{new_topic}

## New Outline:
{new_outline}
"""),
                
            expected_output=dedent("""\
An comprehensive list of cross-references and connections between the new topics.
                """),
            async_execution=False,
            agent=agent, 
        )

    def script_writing_task(self, agent):
        return Task(
            description=dedent("""\
Develop a detailed script based on the provided outline. Ensure the script
is engaging, informative, and aligns with the overall tone and style of the channel.
Incorporate any relevant research findings and expert opinions to add depth and credibility.

##Topic 
{new_topic}

## Outline
{outline}

## Channel Subject
{subject}
            """),
            expected_output=dedent("""\
                A complete and polished script ready for production, including all necessary dialogue, transitions,
                and references to research and expert opinions, using markdown format."""),
            async_execution=False,
            agent=agent,
        )

    def visual_asset_task(self, agent):
        return Task(
            description=dedent("""\
Identify and create visual assets required for the video based on the script.
This includes images, graphics, charts, and any other visual elements that
will enhance the storytelling and provide clear, engaging content for viewers.

## Script 
{script}
                               
## Channel Subject
{subject}
            """),
            expected_output=dedent("""\
A collection of prompts to create visual assets, including images, graphics, charts, and other visual elements, ready to be
incorporated into the video production process.
            """),   
            async_execution=False,
            agent=agent,
            
        )
    
    def create_twitter_post_task(self, agent, video_url):
        return Task(
            description=dedent(f"""\
Create an engaging Twitter post to promote the newly published video. The post should include a compelling
description, relevant hashtags, and a link to the video. Respect the 140 character limit and use appropriate hashtags. 

## Video Title                               

## Video URL
{video_url}

## Channel Subject

{{subject}}
"""),
            expected_output=dedent("""\
A ready-to-publish Twitter post that includes an engaging description, relevant hashtags, and a link to the video.
Respect the 140 character limit and use appropriate hashtags.
"""),
            async_execution=False,
            agent=agent
        )

    def create_linkedin_post_task(self, agent, video_url):
        return Task(
            description=dedent(f"""\
Create an engaging LinkedIn post to promote the newly published video. The post should include a professional
description, relevant hashtags, and a link to the video. Consider the LinkedIn audience and tailor the content
to suit a professional network.

## Video URL
{video_url}
## Channel Subject
{{subject}}
            """),
            expected_output=dedent("""\
A ready-to-publish LinkedIn post that includes a professional description, relevant hashtags, and a link to the video."""),
            async_execution=False,
            agent=agent
        )

    

This class defines various tasks related to content preparation, such as research, outline creation, script writing, visual asset creation, and social media promotion.

## Tools

### Arxiv Research Task

In [17]:
"""Tool for the Arxiv API."""

from typing import Optional, Type

from langchain_core.callbacks import CallbackManagerForToolRun
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain_core.tools import BaseTool

from langchain_community.utilities.arxiv import ArxivAPIWrapper


class ArxivInput(BaseModel):
    """Input for the Arxiv tool."""
    query: str = Field(description="search query to look up, max 5 words")


# you have ArxivQueryRun: taking as arguments: query and max_results
#"A wrapper around Arxiv.org "
#"Useful for when you need to answer questions about Artificial Inteligence, Machine Physics, Mathematics, "
#"Computer Science, Quantitative Biology, Quantitative Finance, Statistics, "
#"Electrical Engineering, and Economics "
#"from scientific articles on arxiv.org. "
#"Input should be a search query using parameter query."

# param: query: str = "cat=cs.AI and dying neurons"
# param: max_results: int = 5



class ArxivQueryRun(BaseTool):
    """Tool that searches the Arxiv API."""

    name: str = "arxiv"
    description: str = (
        "A wrapper around Arxiv.org "
        "Useful for when you need to answer questions about Physics, Mathematics, "
        "Computer Science, Quantitative Biology, Quantitative Finance, Statistics, "
        "Electrical Engineering, and Economics "
        "from scientific articles on arxiv.org. "
        "Input should be a query of keywords to search for using parameter query."
        "Only use described arguments"
    )
    api_wrapper: ArxivAPIWrapper = Field(default_factory=ArxivAPIWrapper)
    args_schema: Type[BaseModel] = ArxivInput

    def _run(
        self,
        query: str,
        run_manager: Optional[CallbackManagerForToolRun] = None,
    ) -> str:
        """Use the Arxiv tool."""
        return self.api_wrapper.run(query)
arxiv = ArxivQueryRun(api_wrapper=ArxivAPIWrapper(doc_content_chars_max=512, load_max_docs=2)) 

docs = arxiv.run("cat=cs.AI and dying neurons")
docs

display(Markdown(f"## Research Papers on the Topic\n\n{docs}"))

## Research Papers on the Topic

No good Arxiv Result was found

### Wikipedia Task

In [18]:
from langchain_community.utilities import WikipediaAPIWrapper

class WikipediaQueryRun(BaseTool):
    """Tool that searches the Wikipedia API."""

    name: str = "wikipedia"
    description: str = (
        "A wrapper around Wikipedia. "
        "Useful for when you need to answer general questions about "
        "people, places, companies, facts, historical events, or other subjects. "
        "Input should be a search query. Only use described arguments."
    )
    api_wrapper: WikipediaAPIWrapper

    def _run(
        self,
        query: str,
        run_manager: Optional[CallbackManagerForToolRun] = None,
    ) -> str:
        """Use the Wikipedia tool."""
        return self.api_wrapper.run(query)

wikipedia = WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper(top_k_results=5, doc_content_chars_max=512))
wikipedia_docs = wikipedia.run("backpropagation algorithms")

display(Markdown(f"## Research Papers on the Topic\n\n{wikipedia_docs}"))

## Research Papers on the Topic

Page: Backpropagation
Summary: In machine learning, backpropagation is a gradient estimation method used to train neural network models. The gradient estimate is used by the optimization algorithm to compute the network parameter updates.
It is an efficient application of the Leibniz chain rule (1673) to such networks. It is also known as the reverse mode of automatic differentiation or reverse accumulation, due to Seppo Linnainmaa (1970). The term "back-propagating error correction" was introduced in 1962 

### Setting up tools for agents


In [7]:
from langchain.agents import load_tools
tools = load_tools(
    [wikipedia, arxiv],
) 

tools

[ArxivQueryRun()]

## Agents

In [8]:
class ContentPreparationAgents:
    def __init__(self, llm):
        self.llm = llm

    # Define the agents involved in the content creation process
    def create_researcher(self,tools=[]):
        return Agent(
            role="Research Agent",

            backstory="""\
            Ray was a former investigative journalist who has a knack for uncovering the latest trends and insights.
            After years of digging deep into stories, Ray transitioned into the digital content world, leveraging his skills to help content creators stay ahead of the curve.""",
            
            goal="To provide comprehensive and up-to-date research that helps shape the direction of the channel’s content.",
            llm=self.llm,
            
            tools=tools,
            allow_delegation=False,
			verbose=True,
            full_output=True
        )

    def create_outline_writer(self,tools=[]):
        return Agent(
            role="Outline Creation Agent",
            backstory="""\
            Olivia was once a strategy consultant for major media firms, known for her ability to synthesize complex information into clear, actionable plans.
            She now uses her talents to help content creators map out their content effectively.""",

            goal="To create detailed and engaging outlines that guide the content creation process seamlessly.",
            llm=self.llm,
            tools=tools,
            allow_delegation=False,
			verbose=True,
            full_output=True
    )
    
    def create_content_agent(self, tools=[]):
        return Agent(
            role="Content Integration Agent",

            backstory="""\
            Chris has a background in library sciences and data management.
            His expertise in organizing and cross-referencing large volumes of information makes him perfect for ensuring new content fits well within existing frameworks.""",
            
            goal="To build a cohesive narrative by connecting new content with existing topics.",

            tools=tools,
            llm=self.llm,
            allow_delegation=False,
			verbose=True,
            full_output=True
        )

    def create_script_writer(self,tools=[]):
        return Agent(
        role="Script Writing Agent",
        backstory="""\
            Sam is a former screenwriter with experience in both film and digital media.
            His storytelling skills ensure that every piece of content is engaging and informative.""",

        goal="To craft scripts that are compelling and aligned with the channel's voice and vision.",
        tools=tools,
        llm=self.llm,
        allow_delegation=False,
		verbose=True,
        full_output=True
    )

    def create_visual_asset_creator(self,tools=[]):
        return Agent(
        role="Visual Asset Creation Agent",

        backstory="""\
        Vicky is a graphic designer and visual artist who has worked with numerous brands to create eye-catching visuals.
        Her ability to translate concepts into compelling visuals and their prompt is unparalleled.""",
        
        goal="To create visual assets that enhance the storytelling of the content.",
        tools=tools,
        llm=self.llm,
        allow_delegation=False,
		verbose=True,
        full_output=True
    )

    def create_social_media_agent(self,tools=[]):
        return Agent(
        role="Social Media and Community Engagement Agent",

        backstory="""\
        Casey is a social media strategist who has successfully managed online communities for various brands.
        With expertise in crafting engaging posts and interacting with audiences, Casey ensures that the content reaches and resonates with a wide audience.""",

        goal="To create and schedule engaging posts for Twitter and LinkedIn that promote the new video content effectively.",
        tools=tools,
        llm=self.llm,
        allow_delegation=False,
		verbose=True,
        full_output=True
    )

This class defines various agents involved in content preparation, such as content agent, 
outline writer, researcher, script writer, visual asset creator, and social media agent.

## Crew

In [19]:
from crewai import Crew, Process

tasks = ContentPreparationTasks()
agents = ContentPreparationAgents(llm)

content_agent = agents.create_content_agent(tools)
outline_writer = agents.create_outline_writer(tools)
researcher = agents.create_researcher(tools)
script_writer = agents.create_script_writer(tools)
visual_asset_creator = agents.create_visual_asset_creator(tools)
social_media_agent = agents.create_social_media_agent(tools)

youtuber_crew = Crew(
    agents=[
        researcher,
        content_agent,
        outline_writer,
        researcher,
        script_writer,
        visual_asset_creator,
        social_media_agent
    ], 
    tasks=[
        tasks.research_task(researcher, "AI, Machine Learning, Neural Networks", "Artificial Intelligence"),

        tasks.outline_creation_task(outline_writer, "Backpropagation Algorithms", "Artificial Intelligence"),
            
        tasks.cross_reference_content_task(content_agent),

        tasks.script_writing_task(script_writer),

        tasks.visual_asset_task(visual_asset_creator),

        tasks.create_twitter_post_task(social_media_agent, "https://youtube.com/video"),
        tasks.create_linkedin_post_task(social_media_agent, "https://youtube.com/video")
    ],
    manager=llm,
    memory=True,
    verbose=True,
    output_log_file="interaction_logs.log",
    full_output=True,
    process=Process.sequential,
)

This section initializes the tasks and agents, assigns specific tasks to the respective agents, and creates a crew of agents managed by the language model.


In [20]:
youtuber_crew.kickoff()

 [DEBUG]: == Working Agent: Research Agent
 [INFO]: == Starting Task: Conduct comprehensive research on each topic already covered by my channel. Gather information on recent
news, trends and research papers about the Channel Topic.
Identify potential gaps and new directions for future content.

Topics: AI, Machine Learning, Neural Networks
Channel Subject: Artificial Intelligence


> Entering new CrewAgentExecutor chain...
Thought: To create an up-to-date research report summarizing new trends, gaps, and future directions for Artificial Intelligence content.
Action: arxiv
Action Input: {"query": "AI recent developments 2023"} 

No good Arxiv Result was found

Thought: Since the initial search did not yield relevant results from arXiv, I need to refine my approach to ensure comprehensive coverage of AI, Machine Learning, and Neural Networks.
Action: arxiv
Action Input: 
{
"query": "Artificial Intelligence machine learning neural networks recent developments 2023"
} 

No good Arxiv Resu

KeyboardInterrupt: 

This command starts the process, activating the agents to perform their assigned tasks.